# Fianacial Sentiment Analysis

In [95]:
#Import the libraries
import numpy as np
import pandas as pd
import re
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import one_hot
import keras.utils as ku
from keras import Sequential
from sklearn import metrics
#! pip install textblob

In [3]:
#Load the dataset
df=pd.read_csv("financila_sentiments.csv")
df

,Sentence,Sentiment
0,The GeoSolutions technology will leverage Bene...,positive
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative
2,"For the last quarter of 2010 , Componenta 's n...",positive
3,According to the Finnish-Russian Chamber of Co...,neutral
4,The Swedish buyout firm has sold its remaining...,neutral
...,...,...
5837,RISING costs have forced packaging producer Hu...,negative
5838,Nordic Walking was first used as a summer trai...,neutral
5839,"According shipping company Viking Line , the E...",neutral
5840,"In the building and home improvement trade , s...",neutral


Conclusion= The data has been loaded properly but now we need to create a corpus.

In [13]:
#Creating a corpus
corpus=[]
PS=PorterStemmer()
for i in range(0,len(df)):
    clean=re.sub("[^a-zA-Z]"," ",str(df["Sentence"][i]))
    clean=clean.lower()
    clean=clean.split()
    clean=[PS.stem(word) for word in clean if word not in stopwords.words("english")]
    clean=" ".join(clean)
    corpus.append(clean)

In [18]:
# Treatment of categorical target col
y=df["Sentiment"].map({"positive":0,"neutral":1,"negative":2})

# BOW

In [34]:
#Creating train test spli with BOW
BOW=CountVectorizer()
X=BOW.fit_transform(corpus)
X_train,X_test,y_train,y_test=train_test_split(X,y)

In [46]:
#Model Building
RF=RandomForestClassifier()
RF.fit(X_train,y_train)

RandomForestClassifier()

In [47]:
#Performance parameter
Predicted=RF.predict(X_test)
print("Accuracy is",metrics.accuracy_score(y_test,Predicted))

Accuracy is 0.6427104722792608


Conclusion=The performance is not good enough.

# TFIDF

In [48]:
#Creating train test split with the help of TFIDF
TFIDF=TfidfVectorizer()
X=TFIDF.fit_transform(corpus)
X_train,X_test,y_train,y_test=train_test_split(X,y)

In [49]:
#Model
#Model Building
RF=RandomForestClassifier()
RF.fit(X_train,y_train)

RandomForestClassifier()

In [50]:
#Performance parameter
Predicted=RF.predict(X_test)
print("Accuracy is",metrics.accuracy_score(y_test,Predicted))

Accuracy is 0.6563997262149213


Conclusion=Performance is not good.

# Apply LSTM for better performance

In [55]:
#Creating a train test split
vocab=[one_hot(word,5000) for word in corpus]
X=ku.pad_sequences(vocab)
X_train,X_test,y_train,y_test=train_test_split(X,y)

In [60]:
#Creating a LSTM Model
#Instantiate the model
model=Sequential()
#Embedding layer
model.add(keras.layers.Embedding(5000,1000))
#Adding LSTM Layer
model.add(keras.layers.LSTM(100))
#Adding a dense layer
model.add(keras.layers.Dense(units=3,activation="softmax"))
#Compilation 
model.compile(optimizer="adam",loss="sparse_categorical_crossentropy",metrics="accuracy")
#Fit
model.fit(X_train,y_train,epochs=20)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Epoch 1/20
137/137 [==============================] - 49s 322ms/step - loss: 0.8480 - accuracy: 0.6161
Epoch 2/20
137/137 [==============================] - 44s 320ms/step - loss: 0.5154 - accuracy: 0.7765
Epoch 3/20
137/137 [==============================] - 44s 318ms/step - loss: 0.3543 - accuracy: 0.8357
Epoch 4/20
137/137 [==============================] - 45s 326ms/step - loss: 0.2567 - accuracy: 0.8749
Epoch 5/20
137/137 [==============================] - 46s 333ms/step - loss: 0.2109 - accuracy: 0.8824
Epoch 6/20
137/137 [==============================] - 45s 328ms/step - loss: 0.1852 - accuracy: 0.8961
Epoch 7/20
137/137 [==============================] - 44s 324ms/step - loss: 0.1660 - accuracy: 0.8998
Epoch 8/20
137/137 [==============================] - 44s 321ms/step - loss: 0.1482 - accuracy: 0.9012
Epoch 9/20
137/137 [==============================] - 44s 323ms/step - loss: 0.1497 - accuracy: 0.9034
Epoch 10/20
137/137 [==============================] - 44s 322ms/step - l

In [67]:
#Model Evaulation on training and testing
print("Traning evalutaion",model.evaluate(X_train,y_train))

137/137 [==============================] - 10s 76ms/step - loss: 0.1122 - accuracy: 0.9265
Traning evalutaion [0.11221194267272949, 0.9265007972717285]


# Thank you